```
# Copyright 2022 by the author(s) of CHI2023 submission "Short-Form
# Videos Degrade Our Capacity to Retain Intentions: Effect of Context
# Switching On Prospective Memory". All rights reserved.
#
# Use of this source code is governed by a GPLv3 license that
# can be found in the LICENSE file.
```

In [9]:
library(lme4)
library(dplyr)
library(bayestestR)
library(report)

In [10]:
df <- read.csv("data/rt.csv")
head(df)

,folder_id,task,interrupt,measure,stimulus,success,correct,rt
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,101,LD,rest,train,TRINKEN,True,True,0.7375202
2,101,LD,rest,train,EIRLHCH,True,True,0.8492210
3,101,LD,rest,train,PLATZ,True,True,1.2250568
4,101,LD,rest,train,ALEEINL,True,True,0.8843554
5,101,LD,rest,train,TEHS,True,True,0.7557791
6,101,LD,rest,train,LERNEN,True,True,0.5365034


## PM Task

In [11]:
df_pm_correct <- df %>% filter((task %in% c('PM'))) %>% filter((success %in% c('True'))) %>% filter((correct %in% c('True'))) %>% filter(!(measure %in% c('train')))

In [12]:
df_pm_correct$interrupt <- factor(df_pm_correct$interrupt, levels=c('rest', 'twitter', 'youtube', 'tiktok'))
df_pm_correct$measure   <- factor(df_pm_correct$measure,   levels=c('pre', 'post'))

m0 <- lmer(rt  ~ interrupt + (1|folder_id), data = df_pm_correct)
m1 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id), data = df_pm_correct, family = Gamma(link = "log"))
m2 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id), data = df_pm_correct, family = Gamma(link = "inverse"))
m3 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id), data = df_pm_correct, family = inverse.gaussian(link = "inverse"))
m4 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id) + (1|stimulus), data = df_pm_correct, family = Gamma(link = "log"))
m5 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id) + (1|stimulus), data = df_pm_correct, family = Gamma(link = "inverse"))
m6 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id) + (1|stimulus), data = df_pm_correct, family = inverse.gaussian(link = "inverse"))

anova(m0, m1, m2, m3, m4, m5, m6)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00235194 (tol = 0.002, component 1)”
refitting model(s) with ML (instead of REML)



,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
m0,6,855.589646,886.88109,-421.79482,843.58965,NA,NA,NA
m1,10,416.362081,468.51448,-198.18104,396.36208,447.22757,4,1.726651e-95
m2,10,417.160226,469.31263,-198.58011,397.16023,0.00000,0,NA
m3,10,325.621815,377.77422,-152.81091,305.62182,91.53841,0,NA
m4,11,97.997415,155.36505,-37.99871,75.99742,229.62440,1,7.198714e-52
m5,11,108.351289,165.71893,-43.17564,86.35129,0.00000,0,NA
m6,11,-4.318017,53.04962,13.15901,-26.31802,112.66931,0,NA


In [13]:
bayesfactor_models(m1, m2, m3, m4, m5, m6, denominator = m0)

,Model,log_BF
,<chr>,<dbl>
m1,interrupt * measure + interrupt + measure + (1 | folder_id),209.2529
m2,interrupt * measure + interrupt + measure + (1 | folder_id),208.8538
m3,interrupt * measure + interrupt + measure + (1 | folder_id),254.6230
m4,interrupt * measure + interrupt + measure + (1 | folder_id) + (1 | stimulus),365.8276
m5,interrupt * measure + interrupt + measure + (1 | folder_id) + (1 | stimulus),360.6507
m6,interrupt * measure + interrupt + measure + (1 | folder_id) + (1 | stimulus),416.9853
m0,interrupt + (1 | folder_id),0.0000


In [14]:
summary(m4)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( log )
Formula: rt ~ interrupt * measure + interrupt + measure + (1 | folder_id) +  
    (1 | stimulus)
   Data: df_pm_correct

     AIC      BIC   logLik deviance df.resid 
    98.0    155.4    -38.0     76.0     1349 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0986 -0.6061 -0.2025  0.3880  5.7110 

Random effects:
 Groups    Name        Variance Std.Dev.
 folder_id (Intercept) 0.010393 0.10195 
 stimulus  (Intercept) 0.002005 0.04478 
 Residual              0.050752 0.22528 
Number of obs: 1360, groups:  folder_id, 60; stimulus, 3

Fixed effects:
                               Estimate Std. Error t value Pr(>|z|)  
(Intercept)                   2.331e-01  1.063e-01   2.194   0.0283 *
interrupttwitter             -3.224e-02  8.334e-02  -0.387   0.6989  
interruptyoutube              1.986e-02  8.192e-02   0.242   0.8085  
interrupttiktok              -4.2

In [15]:
print(report(m4), verbose=TRUE)

We fitted a general linear mixed model (Gamma family with a log link)
(estimated using ML and Nelder-Mead optimizer) to predict rt with interrupt and
measure
 (formula: rt ~ interrupt * measure + interrupt + measure). The model included
folder_id and stimulus
 as random effects (formula: list(~1 | folder_id, ~1 | stimulus)). The model's
total explanatory power is moderate (conditional R2 = 0.21) and the part
related to the fixed effects alone (marginal R2) is of 0.02. The model's
intercept, corresponding to interrupt = rest and measure = pre
, is at 0.23 (95% CI [0.02, 0.44], t(1349) = 2.19, p = 0.028). Within this
model:

  - The effect of interrupt [twitter] is statistically non-significant and
negative (beta = -0.03, 95% CI [-0.20, 0.13], t(1349) = -0.39, p = 0.699; Std.
beta = -0.03, 95% CI [-0.20, 0.13])
  - The effect of interrupt [youtube] is statistically non-significant and
positive (beta = 0.02, 95% CI [-0.14, 0.18], t(1349) = 0.24, p = 0.808; Std.
beta = 0.02, 95% CI [-0.14,

## LD Task

In [16]:
df_ld_correct <- df %>% filter((task %in% c('LD'))) %>% filter((success %in% c('True'))) %>% filter((correct %in% c('True'))) %>% filter(!(measure %in% c('train')))
df_ld_correct$interrupt <- factor(df_ld_correct$interrupt, levels=c('rest', 'twitter', 'youtube', 'tiktok'))
df_ld_correct$measure   <- factor(df_ld_correct$measure,   levels=c('pre', 'post'))

m0 <- lmer(rt  ~ interrupt + (1|folder_id), data = df_ld_correct)
m1 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id), data = df_ld_correct, family = Gamma(link = "log"))
m2 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id), data = df_ld_correct, family = Gamma(link = "inverse"))
m3 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id), data = df_ld_correct, family = inverse.gaussian(link = "inverse"))
m4 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id) + (1|stimulus), data = df_ld_correct, family = Gamma(link = "log"))
m5 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id) + (1|stimulus), data = df_ld_correct, family = Gamma(link = "inverse"))
m6 <- glmer(rt ~ interrupt * measure + interrupt + measure + (1|folder_id) + (1|stimulus), data = df_ld_correct, family = inverse.gaussian(link = "inverse"))

anova(m0, m1, m2, m3, m4, m5, m6)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0132157 (tol = 0.002, component 1)”
refitting model(s) with ML (instead of REML)



,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
m0,6,6184.038,6230.981,-3086.019,6172.038,NA,NA,NA
m1,10,-3442.307,-3364.069,1731.154,-3462.307,9634.34476,4,0.00000e+00
m2,10,-3473.944,-3395.705,1746.972,-3493.944,31.63659,0,NA
m3,10,-4978.996,-4900.757,2499.498,-4998.996,1505.05228,0,NA
m4,11,-6184.898,-6098.835,3103.449,-6206.898,1207.90185,1,1.16953e-264
m5,11,-6349.078,-6263.015,3185.539,-6371.078,164.17986,0,NA
m6,11,-7625.126,-7539.064,3823.563,-7647.126,1276.04821,0,NA


In [17]:
bayesfactor_models(m1, m2, m3, m4, m5, m6, denominator = m0)

,Model,log_BF
,<chr>,<dbl>
m1,interrupt * measure + interrupt + measure + (1 | folder_id),4797.595
m2,interrupt * measure + interrupt + measure + (1 | folder_id),4813.413
m3,interrupt * measure + interrupt + measure + (1 | folder_id),5565.939
m4,interrupt * measure + interrupt + measure + (1 | folder_id) + (1 | stimulus),6164.978
m5,interrupt * measure + interrupt + measure + (1 | folder_id) + (1 | stimulus),6247.068
m6,interrupt * measure + interrupt + measure + (1 | folder_id) + (1 | stimulus),6885.092
m0,interrupt + (1 | folder_id),0.000


In [18]:
summary(m4)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( log )
Formula: rt ~ interrupt * measure + interrupt + measure + (1 | folder_id) +  
    (1 | stimulus)
   Data: df_ld_correct

     AIC      BIC   logLik deviance df.resid 
 -6184.9  -6098.8   3103.4  -6206.9    18458 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3880 -0.5802 -0.2274  0.2892 11.6632 

Random effects:
 Groups    Name        Variance Std.Dev.
 stimulus  (Intercept) 0.005965 0.07723 
 folder_id (Intercept) 0.006117 0.07821 
 Residual              0.085819 0.29295 
Number of obs: 18469, groups:  stimulus, 316; folder_id, 60

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                  -0.256498   0.057769  -4.440 8.99e-06 ***
interrupttwitter             -0.038286   0.078639  -0.487   0.6264    
interruptyoutube              0.095409   0.077581   1.230   0.2188    
interrupttiktok           

In [19]:
print(report(m4), verbose=TRUE)

We fitted a general linear mixed model (Gamma family with a log link)
(estimated using ML and Nelder-Mead optimizer) to predict rt with interrupt and
measure
 (formula: rt ~ interrupt * measure + interrupt + measure). The model included
folder_id and stimulus
 as random effects (formula: list(~1 | folder_id, ~1 | stimulus)). The model's
total explanatory power is moderate (conditional R2 = 0.16) and the part
related to the fixed effects alone (marginal R2) is of 0.04. The model's
intercept, corresponding to interrupt = rest and measure = pre
, is at -0.26 (95% CI [-0.37, -0.14], t(18458) = -4.44, p < .001). Within this
model:

  - The effect of interrupt [twitter] is statistically non-significant and
negative (beta = -0.04, 95% CI [-0.19, 0.12], t(18458) = -0.49, p = 0.626; Std.
beta = -0.04, 95% CI [-0.19, 0.12])
  - The effect of interrupt [youtube] is statistically non-significant and
positive (beta = 0.10, 95% CI [-0.06, 0.25], t(18458) = 1.23, p = 0.219; Std.
beta = 0.10, 95% CI [